# Pymaceuticals Data/ Statistical Analysis

### Dependancies

In [18]:
# Key Dependancies 
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib as mpl
import pandas as pd
import scipy.stats as st
import numpy as np 
import os 
import csv

# Due to Darkmode of Jupyter Notebook
plt.style.use('dark_background')
mpl.rcParams["figure.facecolor"] = '1c2021'
mpl.rcParams["axes.facecolor"] = '1c2021'

# Version of MPL
print("Matplotlib version: ",mpl.__version__)

# Loading Data Files 
mouse_metadata_path = "data/mouse_metadata.csv"
study_results_path = "data/study_results.csv"

# Reading Files 
mouse_data = pd.read_csv(mouse_metadata_path)
study_data = pd.read_csv(study_results_path)

# Merging Data
comb_data=pd.merge(mouse_data,study_data, how="outer")
comb_data.head()


Matplotlib version:  3.2.2


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [36]:
# Number of Unique Mice ID's (n unique)
total_mice=comb_data["Mouse ID"].nunique()
print(f'{total_mice} Mice were used in this study.')

# Duplicate Mice ID's with duplicate timepoints 
dup_mice = comb_data.loc[comb_data.duplicated(subset =["Mouse ID", "Timepoint"]), "Mouse ID"].unique()
dup_mice

# Obtaining data for duplicates
comb_data.loc[comb_data["Mouse ID"] =="g989",:]

# New DF removing Duplicate data
clean_data = comb_data[comb_data["Mouse ID"].isin(dup_mice)==False]
clean_data.head()
# Validating Data is Clean
total_mice_clean=clean_data["Mouse ID"].nunique()
print(f'{total_mice_clean} Mice were used in this study.')

249 Mice were used in this study.
248 Mice were used in this study.


In [39]:
# Objective: Creating a stat table of mean, median, variance, stand dev and 
# SEM of tumor volume for each drug 

# Grouping by Drug 
drug_group = clead_data.groupby("Drug Regimen")
# Mean Tumour Volume
tumour_volume_mean = drug_group['Tumour Volume (mm3)'].mean()
# Median Tumour Volume
#tumour_volume_median = drug_group['Tumour Volume (mm3)'].median()
# Variance of Tumour Volume
#tumour_volume_vari = drug_group['Tumour Volume (mm3)'].var()
# Standard Deviation of Tumour Volume
#tumour_volume_dev = drug_group['Tumour Volume (mm3)'].std()
# Unibiased SEM
tumour_volume_sem = drug_group['Tumour Volume (mm3)'].sem()

# Summary Table Data Frame
summary_stats=pd.DataFrame({ 
    'Mean': tumour_volume_mean,
    #'Median' : tumour_volume_median,
    #'Variance' : tumour_volume_vari,
    #'Standard Deviation' : tumour_volume_dev,
    'Standard Error' : tumour_volume_sem})

# Format to 2dp and display
summary_stats.round(2)

KeyError: 'Column not found: Tumour Volume (mm3)'